In [1]:
from scipy.io import loadmat
mat = loadmat('test.mat')
audio_data=mat['audio_data']
audio_Fs=1000

In [2]:
def butterworth_low_pass_filter(original_signal,order,cutoff,sampling_frequency, figure=False):
    from scipy import signal
    import numpy as np
    b, a = signal.butter(order, 2*cutoff/sampling_frequency, 'low')
    low_pass_filtered_signal = signal.filtfilt(b, a, np.squeeze(original_signal),padlen=3*max(len(a), len(b))-3)
    return low_pass_filtered_signal

def butterworth_high_pass_filter(original_signal,order,cutoff,sampling_frequency, figures=False):
    from scipy import signal
    import numpy as np
    b, a = signal.butter(order, 2*cutoff/sampling_frequency, 'high')
    high_pass_filtered_signal = signal.filtfilt(b, a,np.squeeze(original_signal),padlen=3*max(len(a), len(b))-3)
    return high_pass_filtered_signal

#not tested
def schmidt_spike_removal(original_signal,fs,figures=False):
    from statistics import median
    import numpy as np 
    #Find the window size
    windowsize = round(fs/2)
    #Find any samples outside of a integer number of windows
    trailingsamples = len(original_signal)%windowsize
    #Reshape the signal into a number of windows
    sampleframes=[original_signal[i*windowsize:(windowsize)+i*windowsize] for i in range((len(original_signal)-(trailingsamples))//windowsize)]
    sampleframes=np.array(sampleframes)
    #Find the MAAs

    MAAs = [ max([abs(number) for number in individual_frame]) for individual_frame in sampleframes]
    while any(MAA >median(MAAs)*3 for MAA in MAAs):#Find the window with the max MAA
        window_num=np.argmax(MAAs)
        #Find the postion of the spike within that window
        abssampleframe=[np.abs(number) for number in sampleframes[window_num]]
        spike_position=np.argmax(abssampleframe)
        #Finding zero crossings (where there may not be actual 0 values, just a change from positive to negative)
        selected_window=sampleframes[window_num]
        zero_crossings=np.abs(np.diff(np.sign(selected_window)))
        zero_crossings=np.append(zero_crossings,0)
        zero_crossings=(zero_crossings>1).astype(int)
        #Find the start of the spike, finding the last zero crossing before
        #spike position. If that is empty, take the start of the window:
        zero_crossings_list=zero_crossings.tolist()
        start_nonzeros=[i for i, e in enumerate(zero_crossings_list[:spike_position]) if e != 0]
        if not start_nonzeros:
            spike_start=0
        else:
            spike_start=start_nonzeros[-1]

        #Find the end of the spike, finding the first zero crossing after
        #spike position. If that is empty, take the end of the window:
        zero_crossings_list[:spike_position] = [0] * (spike_position)
        after_nonzeros=[i for i, e in enumerate(zero_crossings_list) if e != 0]
        if not after_nonzeros:
            spike_end=windowsize
        else:
            spike_end=after_nonzeros[-1]

        #Set to Zero
        sampleframes[window_num][spike_start:spike_end] = 0.0001

        #Recaclulate MAAs
        MAAs = [ max([abs(number) for number in individual_frame]) for individual_frame in sampleframes]

    despiked_signal = sampleframes

    # Add the trailing samples back to the signal:
    despiked_signal=np.append(despiked_signal,np.array(original_signal[despiked_signal.size:-1]))

    return despiked_signal
def Homomorphic_Envelope_with_Hilbert(input_signal, samplingFrequency,lpf_frequency=8,figures=False):
    #8Hz, 1st order, Butterworth LPF
    from scipy import signal
    import numpy as np
    B_low,A_low = signal.butter(1,2*lpf_frequency/samplingFrequency,'low')
    hilbert_coeff=np.array(signal.hilbert(input_signal))
    homomorphic_envelope = np.exp(signal.filtfilt(B_low,A_low,np.log(np.abs(hilbert_coeff))))
    #Remove spurious spikes in first sample:
    homomorphic_envelope[0] = homomorphic_envelope[1]

    return homomorphic_envelope

In [3]:
audio_data=audio_data
audio_data = butterworth_low_pass_filter(audio_data,2,400,audio_Fs)
audio_data = butterworth_high_pass_filter(audio_data,2,25,audio_Fs)
# Spike removal from the original paper:
audio_data = schmidt_spike_removal(audio_data,audio_Fs)
homomorphic_envelope=Homomorphic_Envelope_with_Hilbert(audio_data, audio_Fs)

In [7]:
#compare after spike difference values 
mat = loadmat('audio_afterspike.mat')
audio_data_afterspike=mat['audio_data']

In [ ]:
import numpy as np
np.abs(audio_data-audio_data_afterspike)